## Yolo V3 with Tensorflow

In [1]:
from net.yolo_top import yolov3
from data.data_pipeline import data_pipeline
from net.config import cfg
import numpy as np
import time
import tensorflow as tf
import os
% load_ext autoreload
% autoreload 2

In [2]:
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

In [3]:
def tower_loss(scope, imgs, true_boxes, istraining):
    """Calculate the total loss on a single tower running the CIFAR model.
    Args:
        scope: unique prefix string identifying the CIFAR tower, e.g. 'tower_0'
        images: Images. 4D tensor of shape [batch_size, height, width, 3].
        labels: Labels. 1D tensor of shape [batch_size].
    Returns:
        Tensor of shape [] containing the total loss for a batch of data
    """

    # Build inference Graph.
    # Build the portion of the Graph calculating the losses. Note that we will
    # assemble the total_loss using a custom function below.
    model = yolov3(imgs, true_boxes, istraining)
    _ = model.compute_loss()

    # Assemble all of the losses for the current tower only.
    losses = tf.get_collection('losses', scope)

    # Calculate the total loss for the current tower.
    total_loss = tf.add_n(losses, name='total_loss')
    
#    print("loss")
#    print(total_loss)
    
    return total_loss

def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
        tower_grads: List of lists of (gradient, variable) tuples. The outer list
        is over individual gradients. The inner list is over the gradient
        calculation for each tower.
    Returns:
        List of pairs of (gradient, variable) where the gradient has been averaged
        across all towers.
    """
    average_grads = []

    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        
#        print(type(grad_and_vars))
#        print(grad_and_vars)
        for g, _ in grad_and_vars:
#            print("###")
#            print(type(g))
#            print(g)
            
            
            
            
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
def train(num_gpus):

    with tf.Graph().as_default(), tf.device('/cpu:0'):
        # Create a variable to count the number of train() calls. This equals the
        # number of batches processed * FLAGS.num_gpus.
        global_step = tf.Variable(0, trainable=False)
        
        decay_steps = int(100 * 100)

        #lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE, 
        #                                 global_step,
        #                                 decay_steps,
        #                                 LEARNING_RATE_DECAY_FACTOR,
        #                                 staircase=True)

        lr = tf.train.piecewise_constant(global_step, 
                                         cfg.train.lr_steps,
                                         cfg.train.lr_scales)
        
        # Create an optimizer that performs gradient descent.
        
#        opt = tf.train.GradientDescentOptimizer(lr)
        
        opt = tf.train.AdamOptimizer(learning_rate = lr)
        
        tower_grads = []
        
        imgs, true_boxes = data_pipeline(
            [cfg.path.train_data_path, cfg.path.train_data_path], cfg.batch_size)
        
        print(imgs.shape)
        print(true_boxes.shape)
        
        imgs = tf.reshape(imgs, 
                          [cfg.batch_size, imgs.shape[1], imgs.shape[2], imgs.shape[3]])
        true_boxes = tf.reshape(true_boxes, 
                                [cfg.batch_size, true_boxes.shape[1], true_boxes.shape[2]])
        
        batch_queue = tf.contrib.slim.prefetch_queue.prefetch_queue(
            [imgs, true_boxes], capacity=2 * num_gpus)
        
        imgs_batch = [[], []]
        true_boxes_batch = [[], []]
        
        imgs = [[], []]
        targets = [[], []]
        
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(num_gpus):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                        istraining = tf.constant(True, tf.bool)
                        print(scope)
                        
                        imgs_batch[i], true_boxes_batch[i] = batch_queue.dequeue()
                        # Calculate the loss for one tower of the CIFAR model. This function
                        # constructs the entire CIFAR model but shares the variables across
                        # all towers.
                        
                        imgs[i] = tf.placeholder(tf.float32, 
                                              shape = [None, 608, 608, 3], 
                                              name = "input_x")
                        targets[i] = tf.placeholder(tf.float32,
                                                 shape = [None, 30, 5], 
                                                 name = "input_y")
                        
                        loss = tower_loss(scope, 
                                          imgs[i], 
                                          targets[i],
                                          istraining)

                        # Reuse variables for the next tower.
                        tf.get_variable_scope().reuse_variables()

                        # Retain the summaries from the final tower.
                        summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

                        # Calculate the gradients for the batch of data on this CIFAR tower.
                        grads = opt.compute_gradients(loss)
                        
                        print('ops________________________________________________')
                        for i in grads:
                            print(i[0])
                        print ('variables________________________________________________')
                        for i in grads:
                            print (i[1])

                        # Keep track of the gradients across all towers.
                        tower_grads.append(grads)

        # We must calculate the mean of each gradient. Note that this is the
        # synchronization point across all towers.

        grads = average_gradients(tower_grads)
        
        print('average ops________________________________________________')
        for i in grads:
            print(i[0])
        print ('average variables________________________________________________')
        for i in grads:
            print (i[1])

        # Add a summary to track the learning rate.
        summaries.append(tf.summary.scalar('learning_rate', lr))

        # Add histograms for gradients.
        for grad, var in grads:
            if grad is not None:
                summaries.append(tf.summary.histogram(var.op.name + '/gradients', grad))

        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
        
        #update_op = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        #vars_det = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Head")
        
        #with tf.control_dependencies(update_op):
        #    apply_gradient_op = opt.minimize(loss, 
        #                                     global_step = global_step, 
        #                                     var_list = vars_det)
        
        # Add histograms for trainable variables.
        for var in tf.trainable_variables():
            summaries.append(tf.summary.histogram(var.op.name, var))

        # Group all updates to into a single train op.
        train_op = apply_gradient_op

        # Create a saver.
        saver = tf.train.Saver(tf.global_variables())

        # Build the summary operation from the last tower summaries.
        summary_op = tf.summary.merge(summaries)

        # Build an initialization operation to run below.
        init = tf.global_variables_initializer()

        # Start running operations on the Graph. allow_soft_placement must be set to
        # True to build towers on GPU, as some of the ops do not have GPU
        # implementations.
        sess = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False))
        sess.run(init)
        
        # Start the queue runners.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)

        for step in range(10000):
            start_time = time.time()
            
            all_imgs = sess.run(imgs_batch)
            all_true_boxes = sess.run(true_boxes_batch)
            

            _ = sess.run(apply_gradient_op,
                                     feed_dict={imgs[0]: all_imgs[0], targets[0]: all_true_boxes[0], imgs[1]: all_imgs[1], targets[1]: all_true_boxes[1]})
            duration = time.time() - start_time
            
            print(duration)
            if (step % 10 == 0):
                #print(loss_value)
                pass

        for i in range(100):
            start_time = time.time()
            imgs = sess.run(imgs_batch[0])
            duration = time.time() - start_time
            print(duration)
            print(np.array(imgs).shape)

        print("Complete!!")
        coord.request_stop()
        coord.join(threads)
        sess.close()
            
            

In [ ]:
train(2)

(?, 608, 608, 3)
(?, 30, 5)
tower_0/
build start
Tensor("tower_0/Feature_Extractor/conv_0/LeakyRelu/Maximum:0", shape=(?, 608, 608, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_1/LeakyRelu/Maximum:0", shape=(?, 304, 304, 64), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_2/LeakyRelu/Maximum:0", shape=(?, 304, 304, 32), dtype=float32, device=/device:GPU:0)
build done


/home/tsimage-x/anaconda3/envs/tensorflow-work/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


ops________________________________________________
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/sub_grad/tuple/control_dependency:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/mul_grad/tuple/control_dependency_1:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/cond/Switch_1_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/cond/Switch_2_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_1/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/t

build done
ops________________________________________________
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/batchnorm/sub_grad/tuple/control_dependency:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/batchnorm/mul_grad/tuple/control_dependency_1:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/cond/Switch_1_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/cond/Switch_2_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_1/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/

average ops________________________________________________
Tensor("Mean:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_1:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_2:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_3:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_4:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_5:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:CPU:0)
Tensor("Mean_6:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_7:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_8:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_9:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_10:0", shape=(1, 1, 64, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_11:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_12:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_13:

13.422718524932861
0.7336990833282471
0.7380404472351074
0.7615137100219727
0.771925687789917
0.7446081638336182
0.7837893962860107
0.7704133987426758
0.7367119789123535
0.7890148162841797
0.7489383220672607
0.7387092113494873
0.7584302425384521
0.7211201190948486
0.7898757457733154
0.7892246246337891
0.7445597648620605
0.7609632015228271
0.7341170310974121
0.7711043357849121
0.78151535987854
0.7305412292480469
0.7443225383758545
0.7376155853271484
0.7402515411376953
0.7649846076965332
0.7593457698822021
0.7798850536346436
0.7703273296356201
0.7135286331176758
0.75834059715271
0.772444486618042
0.786984920501709
0.7537391185760498
0.7427365779876709
0.7799825668334961
0.7727875709533691
0.7436075210571289
0.752927303314209
0.7621207237243652
0.7540750503540039
0.7345428466796875
0.7292819023132324
0.7334027290344238
0.772087574005127
0.7458400726318359
0.7755074501037598
0.7498519420623779
0.736436128616333
0.7620687484741211
0.7522110939025879
0.7418348789215088
0.7268235683441162
0.7